In [1]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
my_example_nc_file = '/Users/aitorizquierdo/Desktop/air-quality-madrid/sentinel/S5P_OFFL_L2__NO2____20190108T111459_20190108T125629_06414_01_010202_20190114T133207.nc'

In [3]:
from netCDF4 import Dataset
import numpy as np

In [17]:
fh = Dataset(my_example_nc_file, mode='r')

In [7]:
def walktree(top):
    values = top.groups.values()
    yield values
    for value in top.groups.values():
        for children in walktree(value):
            yield children
print (fh)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: KNMI
    source: Sentinel 5 precursor, TROPOMI, space-borne remote sensing, L2
    summary: TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
    tracking_id: ad305a49-6ac9-4734-9157-64ba8e60480a
    id: S5P_OFFL_L2__NO2____20190108T111459_20190108T125629_06414_01_010202_20190114T133207
    time_reference: 2019-01-08T00:00:00Z
    time_reference_days_since_1950: 25209
    time_reference_julian_day: 2458491.5
    time_reference_seconds_since_1970: 1546905600
    time_coverage_start: 2019-01-08T11:36:34Z
    time_coverage_end: 2019-01-08T12:34:57Z
    time_coverage_duration: PT3503.000S
    time_coverage_resolution: PT1.080S
    orbit: 6414
    references: http://www.tropomi.eu/data-products/nitrogen-dioxide
    processor_version: 1.2.2
    keywords_vocabulary: AGU index terms, http://publications.agu.org/author-resource-center/index-terms/
    keywords: 0345 Pollution, U

In [9]:
for children in walktree(fh):
    for child in children:
        print (child)

<class 'netCDF4._netCDF4.Group'>
group /PRODUCT:
    dimensions(sizes): scanline(3245), ground_pixel(450), corner(4), time(1), polynomial_exponents(6), intensity_offset_polynomial_exponents(1), layer(34), vertices(2)
    variables(dimensions): int32 scanline(scanline), int32 ground_pixel(ground_pixel), int32 time(time), int32 corner(corner), int32 polynomial_exponents(polynomial_exponents), int32 intensity_offset_polynomial_exponents(intensity_offset_polynomial_exponents), int32 layer(layer), int32 vertices(vertices), float32 latitude(time,scanline,ground_pixel), float32 longitude(time,scanline,ground_pixel), int32 delta_time(time,scanline), <class 'str'> time_utc(time,scanline), uint8 qa_value(time,scanline,ground_pixel), float32 nitrogendioxide_tropospheric_column(time,scanline,ground_pixel), float32 nitrogendioxide_tropospheric_column_precision(time,scanline,ground_pixel), float32 nitrogendioxide_tropospheric_column_precision_kernel(time,scanline,ground_pixel), float32 averaging_ker

### By printing the fh.groups we can find the groups in the data, which are called ‘PRODUCT’ and ‘METADATA’. By printing fh.groups[‘PRODUCT’] we find the product has data about the dimension and variables.

In [18]:
print (fh.groups)

print (fh.groups['PRODUCT'])

OrderedDict([('PRODUCT', <class 'netCDF4._netCDF4.Group'>
group /PRODUCT:
    dimensions(sizes): scanline(3245), ground_pixel(450), corner(4), time(1), polynomial_exponents(6), intensity_offset_polynomial_exponents(1), layer(34), vertices(2)
    variables(dimensions): int32 scanline(scanline), int32 ground_pixel(ground_pixel), int32 time(time), int32 corner(corner), int32 polynomial_exponents(polynomial_exponents), int32 intensity_offset_polynomial_exponents(intensity_offset_polynomial_exponents), int32 layer(layer), int32 vertices(vertices), float32 latitude(time,scanline,ground_pixel), float32 longitude(time,scanline,ground_pixel), int32 delta_time(time,scanline), <class 'str'> time_utc(time,scanline), uint8 qa_value(time,scanline,ground_pixel), float32 nitrogendioxide_tropospheric_column(time,scanline,ground_pixel), float32 nitrogendioxide_tropospheric_column_precision(time,scanline,ground_pixel), float32 nitrogendioxide_tropospheric_column_precision_kernel(time,scanline,ground_pixe

In [19]:
print (fh.groups['PRODUCT'].variables.keys())

odict_keys(['scanline', 'ground_pixel', 'time', 'corner', 'polynomial_exponents', 'intensity_offset_polynomial_exponents', 'layer', 'vertices', 'latitude', 'longitude', 'delta_time', 'time_utc', 'qa_value', 'nitrogendioxide_tropospheric_column', 'nitrogendioxide_tropospheric_column_precision', 'nitrogendioxide_tropospheric_column_precision_kernel', 'averaging_kernel', 'air_mass_factor_troposphere', 'air_mass_factor_total', 'tm5_tropopause_layer_index', 'tm5_constant_a', 'tm5_constant_b'])


In [20]:
fh.groups['PRODUCT'].variables['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: seconds since 2010-01-01 00:00:00
    standard_name: time
    axis: T
    long_name: reference time for the measurements
    comment: The time in this variable corresponds to the time in the time_reference global attribute
    _FillValue: -2147483647
path = /PRODUCT
unlimited dimensions: 
current shape = (1,)
filling on

In [22]:
lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
no2 = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0,:,:]
print (lons.shape)
print (lats.shape)
print (no2.shape)

no2[fh.groups['PRODUCT'].variables['qa_value'][0,:,:]<0.75]=np.nan
no2_units = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'].units

(3245, 450)
(3245, 450)
(3245, 450)


In [26]:
df = pd.DataFrame(np.ma.filled(no2))

In [29]:
df.dropna()

,0,1,2,3,4,5,6,7,8,9,...,440,441,442,443,444,445,446,447,448,449


In [30]:
no2

masked_array(
  data=[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],
  fill_value=9.96921e+36,
  dtype=float32)

In [31]:
fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0,:,:]

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=9.96921e+36,
  dtype=float32)

In [32]:
df1 = pd.DataFrame(np.ma.filled(lons))

In [33]:
df2 = pd.DataFrame(np.ma.filled(lats))

In [34]:
df1.head(5)

,0,1,2,3,4,5,6,7,8,9,...,440,441,442,443,444,445,446,447,448,449
0,-142.108246,-142.262253,-142.414215,-142.564178,-142.712189,-142.858322,-143.002609,-143.145081,-143.285812,-143.424820,...,179.047668,178.956772,178.864929,178.772110,178.678299,178.583466,178.487579,178.390625,178.292587,178.193405
1,-142.106461,-142.260712,-142.412903,-142.563095,-142.711349,-142.857712,-143.002213,-143.144928,-143.285873,-143.425095,...,178.996429,178.905487,178.813583,178.720718,178.626862,178.531982,178.436050,178.339050,178.240952,178.141724
2,-142.104370,-142.258850,-142.411285,-142.561722,-142.710205,-142.856796,-143.001541,-143.144470,-143.285645,-143.425095,...,178.945267,178.854279,178.762344,178.669434,178.575516,178.480591,178.384613,178.287567,178.189423,178.090149
3,-142.102371,-142.257095,-142.409775,-142.560455,-142.709167,-142.855988,-143.000961,-143.144119,-143.285522,-143.425201,...,178.893906,178.802872,178.710876,178.617920,178.523956,178.428986,178.332962,178.235870,178.137680,178.038361
4,-142.100616,-142.255585,-142.408493,-142.559402,-142.708359,-142.855408,-143.000610,-143.143997,-143.285614,-143.425507,...,178.842209,178.751114,178.659073,178.566071,178.472061,178.377029,178.280960,178.183807,178.085571,177.986206


In [35]:
df2.head(5)

,0,1,2,3,4,5,6,7,8,9,...,440,441,442,443,444,445,446,447,448,449
0,-55.856110,-55.869850,-55.883205,-55.896191,-55.908817,-55.921097,-55.933041,-55.944656,-55.955956,-55.966949,...,-49.435677,-49.387691,-49.339027,-49.289669,-49.239597,-49.188797,-49.137241,-49.084915,-49.031799,-48.977867
1,-55.917236,-55.931011,-55.944401,-55.957420,-55.970081,-55.982391,-55.994362,-56.006008,-56.017334,-56.028355,...,-49.490913,-49.442879,-49.394169,-49.344761,-49.294640,-49.243786,-49.192181,-49.139805,-49.086632,-49.032646
2,-55.978374,-55.992188,-56.005611,-56.018661,-56.031353,-56.043694,-56.055698,-56.067371,-56.078728,-56.089779,...,-49.546158,-49.498074,-49.449314,-49.399860,-49.349689,-49.298786,-49.247131,-49.194702,-49.141479,-49.087440
3,-56.039497,-56.053345,-56.066803,-56.079887,-56.092609,-56.104980,-56.117016,-56.128719,-56.140106,-56.151184,...,-49.601307,-49.553177,-49.504368,-49.454861,-49.404644,-49.353687,-49.301983,-49.249500,-49.196224,-49.142136
4,-56.100624,-56.114502,-56.127995,-56.141109,-56.153866,-56.166267,-56.178333,-56.190067,-56.201481,-56.212585,...,-49.656425,-49.608250,-49.559391,-49.509834,-49.459564,-49.408558,-49.356800,-49.304268,-49.250938,-49.196793


In [39]:
f = Dataset(my_example_nc_file)

In [40]:
print(f)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.7
    institution: KNMI
    source: Sentinel 5 precursor, TROPOMI, space-borne remote sensing, L2
    summary: TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
    tracking_id: ad305a49-6ac9-4734-9157-64ba8e60480a
    id: S5P_OFFL_L2__NO2____20190108T111459_20190108T125629_06414_01_010202_20190114T133207
    time_reference: 2019-01-08T00:00:00Z
    time_reference_days_since_1950: 25209
    time_reference_julian_day: 2458491.5
    time_reference_seconds_since_1970: 1546905600
    time_coverage_start: 2019-01-08T11:36:34Z
    time_coverage_end: 2019-01-08T12:34:57Z
    time_coverage_duration: PT3503.000S
    time_coverage_resolution: PT1.080S
    orbit: 6414
    references: http://www.tropomi.eu/data-products/nitrogen-dioxide
    processor_version: 1.2.2
    keywords_vocabulary: AGU index terms, http://publications.agu.org/author-resource-center/index-terms/
    keywords: 0345 Pollution, U

In [42]:
print(f.variables.keys())

odict_keys([])


In [43]:
no2_2 = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column']

In [44]:
no2_2

<class 'netCDF4._netCDF4.Variable'>
float32 nitrogendioxide_tropospheric_column(time, scanline, ground_pixel)
    units: mol m-2
    standard_name: troposphere_mole_content_of_nitrogen_dioxide
    long_name: Tropospheric vertical column of nitrogen dioxide
    coordinates: longitude latitude
    ancillary_variables: nitrogendioxide_tropospheric_column_precision air_mass_factor_troposphere air_mass_factor_total averaging_kernel
    multiplication_factor_to_convert_to_molecules_percm2: 6.02214e+19
    _FillValue: 9.96921e+36
path = /PRODUCT
unlimited dimensions: 
current shape = (1, 3245, 450)
filling on

In [45]:
no2_2.dimensions

('time', 'scanline', 'ground_pixel')

In [48]:
time_NO2 = fh.groups['PRODUCT'].variables['time']
scanline_NO2 = fh.groups['PRODUCT'].variables['scanline']
ground_pixel_NO2 = fh.groups['PRODUCT'].variables['ground_pixel']
print(time_NO2)
print(scanline_NO2)
print(ground_pixel_NO2)

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: seconds since 2010-01-01 00:00:00
    standard_name: time
    axis: T
    long_name: reference time for the measurements
    comment: The time in this variable corresponds to the time in the time_reference global attribute
    _FillValue: -2147483647
path = /PRODUCT
unlimited dimensions: 
current shape = (1,)
filling on
<class 'netCDF4._netCDF4.Variable'>
int32 scanline(scanline)
    units: 1
    axis: Y
    long_name: along-track dimension index
    comment: This coordinate variable defines the indices along track; index starts at 0
    _FillValue: -2147483647
path = /PRODUCT
unlimited dimensions: 
current shape = (3245,)
filling on
<class 'netCDF4._netCDF4.Variable'>
int32 ground_pixel(ground_pixel)
    units: 1
    axis: X
    long_name: across-track dimension index
    comment: This coordinate variable defines the indices across track, from west to east; index starts at 0
    _FillValue: -2147483647
path = /PRODUCT
unl

In [50]:
sca_value = scanline_NO2[:]
print(sca_value.shape)
print(sca_value)

(3245,)
[   0    1    2 ... 3242 3243 3244]


In [51]:
ground_pixel_NO2_value = ground_pixel_NO2[:]
print(ground_pixel_NO2.shape)
print(ground_pixel_NO2)

(450,)
<class 'netCDF4._netCDF4.Variable'>
int32 ground_pixel(ground_pixel)
    units: 1
    axis: X
    long_name: across-track dimension index
    comment: This coordinate variable defines the indices across track, from west to east; index starts at 0
    _FillValue: -2147483647
path = /PRODUCT
unlimited dimensions: 
current shape = (450,)
filling on


In [64]:
#lat, lon = fh.groups['PRODUCT'].variables['latitude'], fh.groups['PRODUCT'].variables['longitude']
lon = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
lat = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
print(lat)
print(lon)
print(lat[:])
print(lon[:])

[[-55.85611  -55.86985  -55.883205 ... -49.084915 -49.0318   -48.977867]
 [-55.917236 -55.93101  -55.9444   ... -49.139805 -49.08663  -49.032646]
 [-55.978374 -55.992188 -56.00561  ... -49.194702 -49.14148  -49.08744 ]
 ...
 [ 67.44269   67.526115  67.608376 ...  81.40031   81.36004   81.31865 ]
 [ 67.465935  67.549416  67.63173  ...  81.457405  81.41693   81.37533 ]
 [ 67.48922   67.572754  67.65511  ...  81.51434   81.47366   81.43185 ]]
[[-142.10825  -142.26225  -142.41422  ...  178.39062   178.29259
   178.1934  ]
 [-142.10646  -142.26071  -142.4129   ...  178.33905   178.24095
   178.14172 ]
 [-142.10437  -142.25885  -142.41129  ...  178.28757   178.18942
   178.09015 ]
 ...
 [ -54.52469   -54.4739    -54.423454 ...   36.783745   37.255337
    37.728806]
 [ -54.682148  -54.63188   -54.581955 ...   36.92998    37.40388
    37.879623]
 [ -54.839745  -54.790005  -54.7406   ...   37.07953    37.555763
    38.033787]]
[[-55.85611  -55.86985  -55.883205 ... -49.084915 -49.0318   -48.977

In [65]:
# extract lat/lon values (in degrees) to numpy arrays
latvals = lat[:]
lonvals = lon[:] 

# a function to find the index of the point closest pt
# (in squared distance) to give lat/lon value.
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2 
    # 1D index of minimum dist_sq element
    minindex_flattened = dist_sq.argmin()
    # Get 2D index for latvals and lonvals arrays from 1D index
    return np.unravel_index(minindex_flattened, lats.shape)

iy_min, ix_min = getclosest_ij(latvals, lonvals, 40.415363, -3.707398)
print(iy_min)
print(ix_min)

2644
33


In [67]:
Diox_N = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column']
# Read values out of the netCDF file for temperature and salinity
print('%7.4f %s' % (no2_2[0,iy_min,ix_min], no2_2.units))
#print('%7.4f %s' % (sal[0,0,iy_min,ix_min], sal.units))

 0.0003 mol m-2
